In [45]:
import pandas as pd
from utils.quack import Quack
pd.set_option('display.max_columns', 45)

In [46]:
df = Quack.query("""
select game_id, week, season, play_type, "desc", case when field_goal_result = 'made' then 1 when field_goal_result = 'missed' then 0 end as result,
			kick_distance, kicker_player_id,
			kicker_player_name, wind, temp, roof from pbp where play_type = 'field_goal' 
			and field_goal_result <> 'blocked'
;""")
df.iloc[10].to_dict()

{'game_id': '2000_01_CHI_MIN',
 'week': 1,
 'season': 2000,
 'play_type': 'field_goal',
 'desc': '(1:16) G.Anderson 38 yard field goal is GOOD, Center-M.Palmer, Holder-M.Berger.',
 'result': 1,
 'kick_distance': 38.0,
 'kicker_player_id': '00-0000313',
 'kicker_player_name': 'G.Anderson',
 'wind': nan,
 'temp': nan,
 'roof': 'dome'}

In [47]:
df['result'].mean()

np.float64(0.846930877002864)

In [48]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import yaml
import xgboost as xgb
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    mean_squared_error,
    r2_score,
)

def create_model(df, x_cols, y_col, colsample_bytree=0.5, categorical=False) -> xgb.XGBClassifier:
    data = df.loc[~(df[y_col].isna())].copy(deep=True)
    X, y = data[x_cols], data[y_col]
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )
    model = xgb.XGBClassifier(eval_metric="mlogloss", colsample_bytree=colsample_bytree, missing=np.nan, enable_categorical=True)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    # Evaluate
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy: {accuracy:.2f}")

    # Compute confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    print("Confusion Matrix:")
    print(cm)
    return model


x_cols = ['wind', 'temp', 'kick_distance', 'season']
y_col = ['result']
model = create_model(df,x_cols, y_col[0])
model

Accuracy: 0.84
Confusion Matrix:
[[  73  759]
 [  57 4279]]


,objective,'binary:logistic'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,0.5
,device,None
,early_stopping_rounds,None
,enable_categorical,True
,eval_metric,'mlogloss'


In [49]:
from sklearn.metrics import roc_auc_score, average_precision_score, f1_score, confusion_matrix

y_pred = model.predict(df[x_cols])
y_proba = model.predict_proba(df[x_cols])[:,1]

y_test = df[y_col]
print("ROC-AUC:", roc_auc_score(y_test, y_proba))
print("PR-AUC:", average_precision_score(y_test, y_proba))
print("F1:", f1_score(y_test, y_pred))
print("Confusion matrix:\n", confusion_matrix(y_test, y_pred))


ROC-AUC: 0.8261745359602063
PR-AUC: 0.9622706223398746
F1: 0.9233125571699037
Confusion matrix:
 [[  531  3424]
 [  181 21702]]


In [50]:
df['pred'] = y_proba
df

,game_id,week,season,play_type,desc,result,kick_distance,kicker_player_id,kicker_player_name,wind,temp,roof,pred
0,2000_01_ARI_NYG,1,2000,field_goal,"(:05) C.Blanchard 32 yard field goal is GOOD, ...",1,32.0,00-0001343,C.Blanchard,3.0,80.0,outdoors,0.941613
1,2000_01_BAL_PIT,1,2000,field_goal,"(8:21) M.Stover 23 yard field goal is GOOD, Ce...",1,23.0,00-0015784,M.Stover,8.0,74.0,outdoors,0.996011
2,2000_01_BAL_PIT,1,2000,field_goal,"(:11) M.Stover 26 yard field goal is GOOD, Cen...",1,26.0,00-0015784,M.Stover,8.0,74.0,outdoors,0.974490
3,2000_01_BAL_PIT,1,2000,field_goal,"(15:00) M.Stover 33 yard field goal is GOOD, C...",1,33.0,00-0015784,M.Stover,8.0,74.0,outdoors,0.861807
4,2000_01_BAL_PIT,1,2000,field_goal,"(11:31) K.Brown 45 yard field goal is No Good,...",0,45.0,00-0001980,K.Brown,8.0,74.0,outdoors,0.779740
...,...,...,...,...,...,...,...,...,...,...,...,...,...
25833,2025_13_NYG_NE,13,2025,field_goal,(:02) 36-A.Borregales 28 yard field goal is GO...,1,28.0,00-0040200,A.Borregales,NaN,NaN,outdoors,0.974382
25834,2025_13_NYG_NE,13,2025,field_goal,(4:13) 36-A.Borregales 45 yard field goal is N...,0,45.0,00-0040200,A.Borregales,NaN,NaN,outdoors,0.833038
25835,2025_13_NYG_NE,13,2025,field_goal,(2:17) 36-A.Borregales 23 yard field goal is G...,1,23.0,00-0040200,A.Borregales,NaN,NaN,outdoors,0.996359
25836,2025_13_SF_CLE,13,2025,field_goal,"(:04) 14-M.Gay 25 yard field goal is GOOD, Cen...",1,25.0,00-0035269,M.Gay,20.0,35.0,outdoors,0.993218


In [51]:
def calculate_kicker_ratings(df, k_fast=40, k_slow=8, initial_rating=1500):
    """
    Calculate kicker ratings on two timescales:
    - rating_fast: quickly adapts to recent form / injuries
    - rating_slow: slowly adapts, reflects underlying skill
    
    Parameters:
    -----------
    df : DataFrame with ['kicker_player_id', 'result', 'pred', 'season', 'week']
    k_fast : fast-updating k-factor
    k_slow : slow-updating k-factor
    initial_rating : starting rating for all kickers
    
    Returns:
    --------
    df with 'rating_fast', 'rating_slow', and 'rating_combined'
    """
    
    df = df.sort_values(['season', 'week']).copy()
    
    ratings_fast = {}
    ratings_slow = {}
    
    rating_fast_list = []
    rating_slow_list = []
    rating_combined_list = []
    
    for idx, row in df.iterrows():
        kicker_id = row['kicker_player_id']
        
        # Initialize ratings if new kicker
        if kicker_id not in ratings_fast:
            ratings_fast[kicker_id] = initial_rating
            ratings_slow[kicker_id] = initial_rating
        
        # Current ratings
        r_fast = ratings_fast[kicker_id]
        r_slow = ratings_slow[kicker_id]
        
        # ELO-style expected probabilities
        elo_expected_fast = 1 / (1 + 10 ** ((initial_rating - r_fast) / 400))
        elo_expected_slow = 1 / (1 + 10 ** ((initial_rating - r_slow) / 400))
        
        # Combine with situational probability
        combined_expected_fast = (elo_expected_fast + row['pred']) / 2
        combined_expected_slow = (elo_expected_slow + row['pred']) / 2
        
        actual = row['result']
        
        # Update ratings
        ratings_fast[kicker_id] = r_fast + k_fast * (actual - combined_expected_fast)
        ratings_slow[kicker_id] = r_slow + k_slow * (actual - combined_expected_slow)
        
        rating_fast_list.append(ratings_fast[kicker_id])
        rating_slow_list.append(ratings_slow[kicker_id])
        
        # Optional: combined rating for modeling/prediction
        rating_combined_list.append(0.6 * ratings_fast[kicker_id] + 0.4 * ratings_slow[kicker_id])
    
    df['rating_fast'] = rating_fast_list
    df['rating_slow'] = rating_slow_list
    df['kicker_rating'] = rating_combined_list
    df['rating_fast_prev'] = df.groupby('kicker_player_id')['rating_fast'].shift(1).fillna(initial_rating)
    df['rating_slow_prev'] = df.groupby('kicker_player_id')['rating_slow'].shift(1).fillna(initial_rating)
    
    return df


df2 = calculate_kicker_ratings(df, k_fast=175, initial_rating=1500)

# View current ratings
current_ratings = df2.groupby('kicker_player_name').agg({
    'rating_fast': 'last',
    'rating_slow': 'last',
    'kicker_rating': 'last',
    'kicker_player_id': 'first',
    'season': 'last'
}).sort_values('kicker_rating', ascending=False)
print(current_ratings)

                    rating_fast  rating_slow  kicker_rating kicker_player_id  \
kicker_player_name                                                             
N.Folk              2247.665519  1813.927663    2074.170377       00-0025565   
M. Vanderjagt       2329.862067  1674.446394    2067.695798       00-0016830   
C.Boswell           2237.753162  1788.760507    2058.156100       00-0031136   
J.Sanders           2174.892304  1705.207500    1987.018382       00-0034794   
C.Dicker            2155.611332  1680.757007    1965.669602       00-0037224   
...                         ...          ...            ...              ...   
O.Pochman           1324.852479  1492.995215    1392.109573       00-0020379   
S.Castillo          1304.177359  1498.375108    1381.856459       00-0031385   
N.Freese            1253.144855  1485.370577    1346.035143       00-0031093   
M.Koenen            1206.627926  1481.782126    1316.689606       00-0023234   
K.Brindza           1184.953203  1484.60

In [52]:
pregame_ratings = (
    df2.groupby(['game_id', 'kicker_player_id', 'week', 'season'], as_index=False)
      .agg({
          'kicker_rating': 'first'
      })
)

df_with_rating = pregame_ratings.merge(df)
df_with_rating


,game_id,kicker_player_id,week,season,kicker_rating,play_type,desc,result,kick_distance,kicker_player_name,wind,temp,roof,pred
0,2000_01_ARI_NYG,00-0001343,1,2000,1530.208713,field_goal,"(:05) C.Blanchard 32 yard field goal is GOOD, ...",1,32.0,C.Blanchard,3.0,80.0,outdoors,0.941613
1,2000_01_BAL_PIT,00-0001980,1,2000,1430.766083,field_goal,"(11:31) K.Brown 45 yard field goal is No Good,...",0,45.0,K.Brown,8.0,74.0,outdoors,0.779740
2,2000_01_BAL_PIT,00-0015784,1,2000,1527.265810,field_goal,"(8:21) M.Stover 23 yard field goal is GOOD, Ce...",1,23.0,M.Stover,8.0,74.0,outdoors,0.996011
3,2000_01_BAL_PIT,00-0015784,1,2000,1527.265810,field_goal,"(:11) M.Stover 26 yard field goal is GOOD, Cen...",1,26.0,M.Stover,8.0,74.0,outdoors,0.974490
4,2000_01_BAL_PIT,00-0015784,1,2000,1527.265810,field_goal,"(15:00) M.Stover 33 yard field goal is GOOD, C...",1,33.0,M.Stover,8.0,74.0,outdoors,0.861807
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25833,2025_13_NYG_NE,00-0040200,13,2025,1728.479169,field_goal,(:02) 36-A.Borregales 28 yard field goal is GO...,1,28.0,A.Borregales,NaN,NaN,outdoors,0.974382
25834,2025_13_NYG_NE,00-0040200,13,2025,1728.479169,field_goal,(4:13) 36-A.Borregales 45 yard field goal is N...,0,45.0,A.Borregales,NaN,NaN,outdoors,0.833038
25835,2025_13_NYG_NE,00-0040200,13,2025,1728.479169,field_goal,(2:17) 36-A.Borregales 23 yard field goal is G...,1,23.0,A.Borregales,NaN,NaN,outdoors,0.996359
25836,2025_13_SF_CLE,00-0035269,13,2025,1643.783933,field_goal,"(:04) 14-M.Gay 25 yard field goal is GOOD, Cen...",1,25.0,M.Gay,20.0,35.0,outdoors,0.993218


In [53]:
import duckdb 
con = duckdb.connect("data/nfl.duckdb")

# Register the DataFrame as a DuckDB table
con.register("kicker_ratings", pregame_ratings)

In [54]:
pregame_ratings.loc[pregame_ratings.kicker_player_id == '00-0031492']

,game_id,kicker_player_id,week,season,kicker_rating
6837,2015_01_CAR_JAX,00-0031492,1,2015,1527.084894
6874,2015_02_MIA_JAX,00-0031492,2,2015,1490.177243
6895,2015_03_JAX_NE,00-0031492,3,2015,1596.070494
6925,2015_04_JAX_IND,00-0031492,4,2015,1612.129840
6950,2015_05_JAX_TB,00-0031492,5,2015,1506.522499
...,...,...,...,...,...
11866,2025_09_SEA_WAS,00-0031492,9,2025,1795.277856
11869,2025_10_ARI_SEA,00-0031492,10,2025,1812.077555
11909,2025_11_SEA_LA,00-0031492,11,2025,1849.342543
11939,2025_12_SEA_TEN,00-0031492,12,2025,1786.941654


In [55]:

# Now run SQL joining the schedule and depth_chart tables with kicker_ratings
query = """
create or replace table kicker_ratings as
select d.*,
COALESCE(
k.kicker_rating,
        LAST_VALUE(k.kicker_rating) 
            OVER (
                PARTITION BY d.gsis_id 
                ORDER BY d.gameday
                ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING
            )
    ) AS kicker_rating
from depth as d
left join kicker_ratings as k
 on d.game_id = k.game_id
  and d.gsis_id = k.kicker_player_id where position = 'K'
  order by gameday
"""

all_kickers_df = con.execute(query).df()

all_kickers_df
con.close()


In [56]:

# Load tables
p = Quack.query('''select p.*, adv.stuffed,
mediocre, explosive, huge_play, 
first_down, short_first, epa
from player_weekly_agg as p 
left join rusher_adv as adv
on p.gsis_id = adv.gsis_id
and p.game_id = adv.game_id
''')
k = Quack.query("SELECT * FROM kicker_ratings")

k


,gameday,game_id,player_name,team,gsis_id,position,pos_rank,formation,season,week,dense_depth,kicker_rating
0,2001-09-09,2001_01_CAR_MIN,Mitch,MIN,00-0001157,K,2,Special Teams,2001,1,1,NaN
1,2001-09-09,2001_01_SEA_CLE,Phil,CLE,00-0004091,K,1,Special Teams,2001,1,0,1564.619066
2,2001-09-09,2001_01_STL_PHI,David,PHI,00-0000108,K,1,Special Teams,2001,1,0,1662.543590
3,2001-09-09,2001_01_CAR_MIN,Gary,MIN,00-0000313,K,1,Special Teams,2001,1,0,1769.712932
4,2001-09-09,2001_01_WAS_SD,Wade,SD,00-0013699,K,1,Special Teams,2001,1,0,1564.984276
...,...,...,...,...,...,...,...,...,...,...,...,...
14950,2025-12-07,2025_14_PIT_BAL,Tyler Loop,BAL,00-0040074,K,1,Special Teams,2025,14,0,NaN
14951,2025-12-07,2025_14_CHI_GB,Brandon McManus,GB,00-0029822,K,1,Special Teams,2025,14,0,1691.993020
14952,2025-12-07,2025_14_SEA_ATL,Zane Gonzalez,ATL,00-0033862,K,1,Special Teams,2025,14,0,1763.657000
14953,2025-12-07,2025_14_MIA_NYJ,Nick Folk,NYJ,00-0025565,K,1,Special Teams,2025,14,0,2056.668302


In [57]:
p

,dense_depth,team,position,gsis_id,game_id,season,week,season_1,player_name,dense_depth_raw,player_id,player_display_name,opponent,offense_snaps,offense_pct,passing_interceptions_roll,attempts_roll,offense_snaps_roll,air_yards_share_roll,receiving_yards_after_catch_roll,receptions_roll,receiving_tds_roll,...,receiving_air_yards_roll,carries_roll,receiving_first_downs_roll,sacks_suffered_roll,sack_yards_lost_roll,rushing_yards_roll,passing_yards_after_catch_roll,rushing_first_downs_roll,rushing_tds_roll,target_share_roll,targets_roll,completion_pct_roll,yards_per_carry_roll,yards_per_catch_roll,yards_per_target_roll,stuffed,mediocre,explosive,huge_play,first_down,short_first,epa
0,0,DEN,RB,00-0029683,2013_01_BAL_DEN,2013,1,2013,Ronnie,0,00-0029683,Ronnie Hillman,BAL,15.0,0.21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.242885,0.266112,0.069383,0.013662,0.206809,0.518358,-0.089818
1,1,DEN,RB,00-0030522,2013_01_BAL_DEN,2013,1,2013,Montee,2,00-0030522,Montee Ball,BAL,18.0,0.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.167870,0.378135,0.072833,0.016618,0.197262,0.662604,-0.053570
2,2,DEN,RB,00-0026988,2013_01_BAL_DEN,2013,1,2013,Knowshon,3,00-0026988,Knowshon Moreno,BAL,37.0,0.52,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.166929,0.327247,0.062215,0.010865,0.218361,0.894599,-0.047311
3,0,DET,RB,00-0024217,2013_01_MIN_DET,2013,1,2013,Reggie,0,00-0024217,Reggie Bush,MIN,55.0,0.69,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.208760,0.296885,0.121900,0.016297,0.214346,0.607556,-0.036713
4,1,DET,RB,00-0027218,2013_01_MIN_DET,2013,1,2013,Joique,1,00-0027218,Joique Bell,MIN,28.0,0.35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.148551,0.318578,0.130564,0.033499,0.220269,0.767657,0.069813
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80448,2,HOU,WR,00-0029435,2014_03_HOU_NYG,2014,3,2014,Damaris,3,00-0029435,Damaris Johnson,NYG,26.0,0.40,0.0,0.0,6.368291,0.013746,1.676266,0.433086,0.000000,...,5.271469,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.043955,1.101961,NaN,3.000000,3.780150,4.004489,0.247894,0.255027,0.087402,0.019942,0.283824,0.683075,-0.049155
80449,3,HOU,WR,00-0029435,2014_04_BUF_HOU,2014,4,2014,Damaris,4,00-0029435,Damaris Johnson,BUF,20.0,0.29,0.0,0.0,12.469482,-0.023116,4.083450,0.579451,0.000000,...,0.812353,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.059784,1.333815,NaN,3.000000,3.852023,3.892374,0.236311,0.243111,0.083319,0.019011,0.270562,0.683075,-0.043875
80450,2,HOU,WR,00-0029435,2014_11_HOU_CLE,2014,11,2014,Damaris,3,00-0029435,Damaris Johnson,CLE,55.0,0.64,0.0,0.0,28.425957,0.089839,4.398219,1.414969,0.059663,...,24.686649,0.139213,1.173118,0.0,0.0,0.636400,0.0,0.0,0.0,0.085975,2.528795,NaN,3.192030,11.756347,8.895731,0.225627,0.232120,0.079552,0.018151,0.258330,0.683075,-0.040044
80451,2,HOU,WR,00-0029435,2014_13_TEN_HOU,2014,13,2014,Damaris,2,00-0029435,Damaris Johnson,TEN,45.0,0.63,0.0,0.0,36.891447,0.091239,10.179833,2.424120,0.033477,...,25.701960,0.328909,1.097138,0.0,0.0,1.861861,0.0,0.0,0.0,0.129053,3.801483,NaN,3.658537,7.582197,6.031960,0.259553,0.221951,0.076066,0.017356,0.247012,0.683075,-0.101720


In [58]:
pd.concat((p,k))

,dense_depth,team,position,gsis_id,game_id,season,week,season_1,player_name,dense_depth_raw,player_id,player_display_name,opponent,offense_snaps,offense_pct,passing_interceptions_roll,attempts_roll,offense_snaps_roll,air_yards_share_roll,receiving_yards_after_catch_roll,receptions_roll,receiving_tds_roll,...,sack_yards_lost_roll,rushing_yards_roll,passing_yards_after_catch_roll,rushing_first_downs_roll,rushing_tds_roll,target_share_roll,targets_roll,completion_pct_roll,yards_per_carry_roll,yards_per_catch_roll,yards_per_target_roll,stuffed,mediocre,explosive,huge_play,first_down,short_first,epa,gameday,pos_rank,formation,kicker_rating
0,0,DEN,RB,00-0029683,2013_01_BAL_DEN,2013,1,2013.0,Ronnie,0.0,00-0029683,Ronnie Hillman,BAL,15.0,0.21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.242885,0.266112,0.069383,0.013662,0.206809,0.518358,-0.089818,NaN,NaN,NaN,NaN
1,1,DEN,RB,00-0030522,2013_01_BAL_DEN,2013,1,2013.0,Montee,2.0,00-0030522,Montee Ball,BAL,18.0,0.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.167870,0.378135,0.072833,0.016618,0.197262,0.662604,-0.053570,NaN,NaN,NaN,NaN
2,2,DEN,RB,00-0026988,2013_01_BAL_DEN,2013,1,2013.0,Knowshon,3.0,00-0026988,Knowshon Moreno,BAL,37.0,0.52,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.166929,0.327247,0.062215,0.010865,0.218361,0.894599,-0.047311,NaN,NaN,NaN,NaN
3,0,DET,RB,00-0024217,2013_01_MIN_DET,2013,1,2013.0,Reggie,0.0,00-0024217,Reggie Bush,MIN,55.0,0.69,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.208760,0.296885,0.121900,0.016297,0.214346,0.607556,-0.036713,NaN,NaN,NaN,NaN
4,1,DET,RB,00-0027218,2013_01_MIN_DET,2013,1,2013.0,Joique,1.0,00-0027218,Joique Bell,MIN,28.0,0.35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.148551,0.318578,0.130564,0.033499,0.220269,0.767657,0.069813,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14950,0,BAL,K,00-0040074,2025_14_PIT_BAL,2025,14,NaN,Tyler Loop,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-12-07,1,Special Teams,NaN
14951,0,GB,K,00-0029822,2025_14_CHI_GB,2025,14,NaN,Brandon McManus,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-12-07,1,Special Teams,1691.993020
14952,0,ATL,K,00-0033862,2025_14_SEA_ATL,2025,14,NaN,Zane Gonzalez,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-12-07,1,Special Teams,1763.657000
14953,0,NYJ,K,00-0025565,2025_14_MIA_NYJ,2025,14,NaN,Nick Folk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-12-07,1,Special Teams,2056.668302


In [59]:
x_cols = ['wind', 'temp', 'kick_distance', 'season', 'kicker_rating']
y_col = ['result']
model = create_model(df_with_rating,x_cols, y_col[0])
model

Accuracy: 0.85
Confusion Matrix:
[[ 122  674]
 [ 109 4263]]


,objective,'binary:logistic'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,0.5
,device,None
,early_stopping_rounds,None
,enable_categorical,True
,eval_metric,'mlogloss'


In [60]:
import joblib
from sim_utils.modeling import update_config
joblib.dump(model, "models/fg_model.joblib")

feature_config = {
    "fg_cols": x_cols,
}
update_config(feature_config)



In [61]:
# Find the index of each kicker's max rating
idx = df2.groupby('kicker_player_name')['kicker_rating'].idxmax()

# Use that to get the corresponding rating and season/week/game
max_ratings = df2.loc[idx, ['kicker_player_name', 'kicker_rating', 'season', 'week']]

# Optional: sort by rating
max_ratings = max_ratings.sort_values('kicker_rating', ascending=False).reset_index(drop=True)

max_ratings.head(20)


,kicker_player_name,kicker_rating,season,week
0,J.Tucker,2495.288859,2018,16
1,J.Hanson,2233.247789,2009,5
2,M.Bryant,2220.644953,2018,13
3,R.Gould,2212.609142,2019,1
4,A.Vinatieri,2202.129661,2016,9
5,C.Boswell,2190.186347,2025,6
6,S.Gostkowski,2164.947055,2015,11
7,N.Folk,2150.953174,2025,12
8,M.Crosby,2132.510483,2021,5
9,J.Brown,2128.054980,2015,13


In [62]:
max_ratings.loc[max_ratings.kicker_player_name=='B.Aubrey']

,kicker_player_name,kicker_rating,season,week
34,B.Aubrey,2017.20856,2024,11
